In [ ]:
## Run in python 2.7+
# Script for all sensor data

import sys

import arable
from   arable.client     import ArableClient 
a = ArableClient()
from   sensor_includes   import email, password_ccber, tenant_ccber, client_id, client_secret, access_token, folder_id
a.connect(email = email, password = password_ccber, tenant = tenant_ccber)

import re

import matplotlib.pyplot as plt
import numpy             as np
import matplotlib.dates  as mdates

import altair            as alt
alt.renderers.enable('notebook')

from   datetime          import timedelta
from   datetime          import datetime
dt = datetime.now()

import pandas            as     pd
from   pandas.plotting   import register_matplotlib_converters
register_matplotlib_converters()

from   io                import StringIO

import arable_data_dict  as dd

In [ ]:
# pip install boxsdk
# pip install "boxsdk[jwt]"
# OAuth 2.0 with JWT (Server Authentication)
# *config.json - JSON file of app configuration settings
import boxsdk
from boxsdk import JWTAuth
from boxsdk import Client, OAuth2
from boxsdk import DevelopmentClient

# Configure JWT auth object
# sdk = JWTAuth.from_settings_file('562613_bwmlw5of_config.json')
# Get auth client
# client = Client(sdk)
# service_account = client.user().get()
# print('Service Account user ID is {0}'.format(service_account.id))
auth = OAuth2(client_id = client_id, client_secret = client_secret, access_token = access_token,)
client = Client(auth)
me = client.user().get()
print('My user ID is {0}'.format(me.id))
"""
# Set folder values
folder_name = 'TESTING'
folder_id = '0'

# Create folder
folder = client.folder(folder_id=folder_id).create_subfolder(folder_name)
"""

from  sensor_includes  import folder_id
items = client.folder(folder_id = folder_id).get_items()

for item in items:
    print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))

In [ ]:
# Datetime object to format "%Y-%m-%dT%H:%M:%SZ"
def dt_to_ymd_hms(x):
    return x.strftime("%Y-%m-%dT%H:%M:%SZ")
# Append "%Y-%m-%dT%H:%M:%SZ" to list
def append_ymd_hms(a, b):
    a.append(dt_to_ymd_hms(b))
# Reverse dt_to_ymd_hms
def ymd_hms_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S")

# Datetime object to format "%Y-%m-%d"
def dt_to_ymd(x):
    return x.strftime("%Y-%m-%d")
# Reverse dt_to_ymd
def ymd_to_dt(x):
    return datetime.strptime(x, "%Y-%m-%d")

# Datetime object to format "%Y-%m"
def dt_to_ym(x):
    return x.strftime("%Y-%m")
# Append "%Y-%m" to list
def append_ym(a, b):
    try:
        a.append(dt_to_ym(b))
    except:
        a.append(dt_to_ym(ym_to_dt(b)))
# Reverse dt_to_ym
def ym_to_dt(x):
    return datetime.strptime(x, "%Y-%m")

# Append string to list
def append_string(a, b):
    a.append(str(b))
    
# Append temp lists to list
def append_temp_list(temp_list, main_list):
    main_list.append(temp_list)

In [ ]:
def arable_query(s, b, c, d, e, f, g):
    df = a.query(select = str(s),
               format = str(b),
               devices = c,
               measure = str(d),
               order = str(e),
               end = f,
               start = g,
               limit = 5000) #100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df



all_sensors = []
for i in a.devices():
    all_sensors.append(str(i['name']))
all_sensors_old = ['A000522', 'A000536', 'A000502', 'A000563', 'A000507', 'A000229', 'A000513', 'A000465', 'A000318', 'A000369', 'A000466', 'A000546', 'A000514', 'A000417', 'A000461', 'A000543', 'A001512']
#good_sensors_old = ['A000536', 'A000502', 'A000563', 'A000507', 'A000229', 'A000513', 'A000465', 'A000318', 'A000369', 'A000466', 'A000546', 'A000522']
bad_sensors  = ['A000514', 'A000417', 'A000543', 'A001512', 'A000461']
good_sensors_old  = list(set(all_sensors_old) - set(bad_sensors))
good_sensors_old = sorted(good_sensors_old, key=lambda x: int(re.search(r'\d+$',x).group()))
#print(good_sensors_old)

good_sensors  = list(set(all_sensors) - set(bad_sensors))
good_sensors = sorted(good_sensors, key=lambda x: int(re.search(r'\d+$',x).group()))
#print(good_sensors)

test = list(set(all_sensors) - set(all_sensors_old))
test = sorted(test, key=lambda x: int(re.search(r'\d+$',x).group()))
#print(test)

# FOUND NEW SENSOR WITH ID # A000*** ... ERROR - NO DATA AVAILABLE!!
# FOUND NEW SENSOR WITH ID # A000*** ... DATA RECEIVED ... ERROR - SENSOR NOT IN RANGE!!
# FOUND NEW SENSOR WITH ID # A000*** ... DATA RECEIVED ... SENSOR ENABLED!

if len(test) != 0:
    for j in test:
        print('FOUND NEW SENSOR WITH ID #', str(j),)
        for i in a.devices():
            if str(i['name']) == j:
                #Look at arable query to see if latlon in the correct range to count as a "good" sensor.
                created = ymd_hms_to_dt(str(i['created'])[:19])
                last_seen = ymd_hms_to_dt(str(i['last_seen'])[:19])
                try:
                    df = arable_query('all', 
                                      'csv', 
                                      [j], 
                                      "daily", 
                                      "time", 
                                      dt, 
                                      dt_to_ymd_hms(dt - timedelta(days=1)))
                    lat = float(str(df['lat'].values.astype(float)).strip('[ ]'))
                    lon = float(str(df['long'].values.astype(float)).strip('[ ]'))
                    print('... DATA RECEIVED',)
                    if 34.3 <= lat <= 35.07 and -120.610 <= lon <= -119.740:
                        print('... SENSOR ENABLED!')
                        good_sensors.append(j)
                    else:
                        print('... ERROR - SENSOR NOT IN RANGE!!')
                        
                except:
                    print('... ERROR - NO DATA AVAILABLE!!')
else:
    print('NO NEW SENSORS DETECTED.')
    
print('SEARCHING FOR NEW DATA FROM... ')
print(len(good_sensors))
for i in range(len(good_sensors)//4 + 1):
    print('\t'.join(good_sensors[i*4:(i+1)*4]), '\n')


In [ ]:
device_ids     = [] # Ex. A00****
device_names   = [] # Ex. Lagoon Ice Plants
device_created = [] # Ex. 2019-10-10T00:00:00
dic_list       = []
# The devices of interest have Z in the Location Name on the Arable website.
# These Location Names are added to device_names.
# The corresponding Device ID is added to device_ids.


for i in a.devices():
    for j in good_sensors:
        if i['name'] == j:
            device_names.append(str(i['location']['name']))
            device_ids.append(str(i['name']))
            device_created.append(str(i['created'])[:19])
            dic_list_temp = {
                'id' : str(i['name']),
                'name' : str(i['location']['name'])[1:],
                'created' : ymd_hms_to_dt(str(i['created'])[:19]),
                'num' : len(dic_list) + 1
            }
            dic_list.append(dic_list_temp)




print(['{:<7}'.format(str(i)) for i in device_ids])
print(device_names)
print(device_created)
test = pd.DataFrame(dic_list)
print(test)

In [ ]:
dates    = [ymd_hms_to_dt(i) for i in device_created]
print(device_ids)

# Converts each item in device_ids to a list.          
device_ids = list(map(lambda el:[el], device_ids))
print(device_ids)
def arable_query(s, b, c, d, e, f, g):
    df = a.query(select = str(s),
               format = str(b),
               devices = c,
               measure = str(d),
               order = str(e),
               end = f,
               start = g,
               limit = 5000) #100000000)
    df = StringIO(df)
    df = pd.read_csv(df, sep=',', error_bad_lines=False)
    return df

lat = []
lon = []
for i in device_ids:
    df = arable_query('all', 'csv', i, "daily", "time", dt_to_ymd_hms(dt), dt_to_ymd_hms(dt - timedelta(days=1)))
    lat.append(float(str(df['lat'].values.astype(float)).strip('[ ]')))
    lon.append(float(str(df['long'].values.astype(float)).strip('[ ]')))

data_location = {'name': device_ids, 'location': device_names, 'lat': lat, 'lon': lon}
df_location   = pd.DataFrame(data=data_location)
print(df_location)

In [ ]:
# Makes date deployed and location maps
ax1 = alt.Chart(test).mark_circle(size=60).encode(
    x=alt.X('created:T', axis=alt.Axis(title='', format = ("%b %d"), labelAngle=-45), scale=alt.Scale(
            domain=[str(ymd_hms_to_dt('2019-02-01T00:00:00')), str(ymd_hms_to_dt('2019-04-01T00:00:00'))])),
    y=alt.Y('id:O', axis=alt.Axis(title='', labelOpacity=0, tickOpacity=0)),
    color='id',
    tooltip=['name']
).properties(
    title='Date Deployed',
    width=800,
    height=40
).interactive()


ax2 = alt.Chart(df_location).mark_point(size=100, shape='circle').encode(
    x=alt.X('lat:Q', axis=alt.Axis(title=''), scale=alt.Scale(
            domain=[34.3, 35.07])),
    y=alt.Y('lon:Q', axis=alt.Axis(title=''), scale=alt.Scale(
            domain=[-120.610, -119.740])),
    color='name',
    tooltip=['name', 'location']
).properties(
    title='Location',
    width=800,
    height=200
).interactive()



#ax1 | ax2
alt.vconcat(ax1, ax2)

In [ ]:
# Another option for date deployed chart

levels = np.tile([6, -4,  3, -3, 2, -8, 7, -5, 6,  -5, 1, -1],
                 int(np.ceil(len(dates)/12)))[:len(dates)]

fig, ax = plt.subplots(figsize=(10, 3))
ax.set(title="Sensor Startup Date")

markerline, stemline, baseline = ax.stem(dates, levels,
                                         linefmt="k-", basefmt="k-",
                                         use_line_collection=True)

plt.setp(markerline, mec="k", mfc="w", zorder=3)
markerline.set_ydata(np.zeros(len(dates)))

vert = np.array(['top', 'bottom'])[(levels > 0).astype(int)]
for d, l, r, va in zip(dates, levels, device_ids, vert):
    ax.annotate(r, xy=(d, l), xytext=(-3, np.sign(l)*3),
                textcoords="offset points", va=va, ha="right")

# Format X axis with 4 month intervals
ax.get_xaxis().set_major_locator(mdates.DayLocator(interval=5))
ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%d %b %Y"))
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

# Remove Y axis and spines
ax.get_yaxis().set_visible(False)
for spine in ["left", "top", "right"]:
    ax.spines[spine].set_visible(False)

ax.margins(y=0.5)
plt.show()

In [ ]:
# Plot sensor locations on state and county maps
# TO DO - FIX LABELS
import altair as alt
from vega_datasets import data
alt.renderers.enable('notebook')

# Load the data as a dict object
us_10m  = data.us_10m()
# Select the geometries under states under objects, filter for California (id = 6)
us_10m['objects']['states']['geometries']=[item for item in us_10m['objects'] \
      ['states']['geometries'] if item['id'] in [6]]
# Make California topojson data
state = alt.Data(
    values=us_10m, 
    format=alt.TopoDataFormat(feature='states',type='topojson'))
# Make California background
background = alt.Chart(state).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=500,
    height=300
).project('mercator')

# Make SB County topojson data
SB_County = alt.Data(url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/CA/Santa%20Barbara.geo.json', format=alt.DataFormat(property='features',type='json'))
# SB County as background
county = alt.Chart(SB_County).mark_geoshape(
    fill='lightblue',
    stroke='white'
).properties(
    width=500,
    height=300
).project('mercator')

# Plot sensor locations
points = alt.Chart(df_location).mark_point(
    size=1,
    color='black'
).encode(
    longitude='lon:Q',
    latitude='lat:Q'
)

# Label sensor names
text = points.mark_text(
    align='left',
    baseline='bottom',
    dx=9,
    dy=9,
    fontSize=9,
    tooltip={"content": "encoding"}
).encode(
    text='name:O'
)

ca = background + county + points 
sb = county + points + text
#alt.vconcat(ca, sb)
ca | sb

In [ ]:
# Monthly
m_start    = []
m_start_fn = []
m_end      = []
m_end_fn   = []

# Yearly
y_start    = []
y_start_fn = []
y_end      = []
y_end_fn   = []

# All Time
astart     = []
start_fn   = []
end        = []
end_fn     = []

for j in range(len(device_created)):
    ms  = []
    msf = []
    me  = []
    mef = []
    
    ys  = []
    ysf = []
    ye  = []
    yef = []
    
    s   = []
    sf  = []
    e   = []
    ef  = []
    
    def new_month(new_m):
        append_ymd_hms(ms, new_m)
        append_string(me, new_m)
        append_ym(msf, new_m)  
        append_ym(mef, new_m)
        
    def new_year(new_y):
        append_ymd_hms(ys, new_y)
        append_string(ye, new_y)
        append_ym(ysf, new_y)
        append_ym(yef, new_y)
    
    device_created_ymd = (device_created[j])[:10]
    start              = ymd_to_dt(device_created_ymd)
    append_string(ms, start)
    append_string(ys, start)
    append_string(s, start)
    append_ym(msf, start)
    append_ym(ysf, start)
    append_ym(sf, start)
    
    start  = start.replace(day=1)
    
    if start.year == dt.year:
        for i in range(start.month+1, dt.month+1):
            new_month(start.replace(month=i))
    
    elif start.year != dt.year: 
        for i in range(dt.year-start.year):
            for j in range(start.month+1, 13): 
                new_month(start.replace(month=j))
        for i in range(start.year+1, dt.year+1):
            newyr = start.replace(year=i, month=1)
            new_year(newyr)
            for j in range(1, dt.month+1):
                new_month(newyr.replace(month=j))
    
    append_string(me, dt)
    append_string(ye, dt)
    append_string(e, dt)
    append_ym(mef, dt)
    append_ym(yef, dt)
    append_ym(ef, dt)
    
    append_temp_list(ms, m_start)
    append_temp_list(me, m_end)
    append_temp_list(msf, m_start_fn)
    append_temp_list(mef, m_end_fn)
    
    append_temp_list(ys, y_start)
    append_temp_list(ye, y_end)
    append_temp_list(ysf, y_start_fn)
    append_temp_list(yef, y_end_fn)
    
    append_temp_list(s, astart)
    append_temp_list(e, end)
    append_temp_list(sf, start_fn)
    append_temp_list(ef, end_fn)
print(ms)
print(me)

In [ ]:
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status))
    sys.stdout.flush()

In [ ]:
# Writes a .csv file for each device.
# Hourly data is separated by month; daily data is separated by year.
# Ex. A000***_hourly_2019-01_2019-02.csv
import time
import os
def write_csv_file(sta, end, i, sta_f, end_f, device): 
    print(str(device)[2:-2] + ' has been active since ' + str(sta[0]) + '.')
    for j in range(len(sta)):
        fn = str(device)[2:-2] + '_' + str(i) + '_' + str(sta_f[j]) + '_' + str(end_f[j]) + '.csv'
        try:
            df = a.query(select = 'all', 
                         format = 'csv', 
                         devices = device, 
                         measure = str(i), 
                         order = "time", 
                         end = end[j], 
                         start = sta[j], 
                         limit = 100000000)
            df = StringIO(df)
            df = pd.read_csv(df, sep=',', error_bad_lines=False)
            #print(df)
            df = df.drop('location', 1)
            df.to_csv(fn, sep = ",")
            print('Successfully wrote ' + fn)
            box_file = client.folder(folder_id).upload(str(fn), str(fn))
        except:
            print('Error writing ' + fn)
            continue  
        

In [ ]:
#dt2 = dt - timedelta(weeks = 1)

dt2 = (dt - timedelta(weeks = 4)).replace(day=1)
dt = dt.replace(day=1)
print(dt)
print(dt2)
stawk = []
endwk = []
stawk_fn = []
endwk_fn = []
append_string(stawk, dt2)
append_string(endwk, dt)
append_ym(stawk_fn, dt2)
append_ym(endwk_fn, dt)
print(stawk)
print(endwk)
print(stawk_fn)
print(endwk_fn)
print(device_ids)

In [ ]:
# Writes data to a .csv file
# Python 3:
# f_input = raw_input('Write data to a .csv file? (y/n) ')

# Python 2:
f_input = input('Write data to a .csv file? (y/n) ')
if f_input == 'y':
    print('\n Select data type: ')
    print(str('{:<2d}'.format(1) + '{:<8}'.format('Daily')      ))
    print(str('{:<2d}'.format(2) + '{:<8}'.format('Hourly')     ))
    print(str('{:<2d}'.format(3) + '{:<8}'.format('Soil')       ))
    print(str('{:<2d}'.format(4) + '{:<8}'.format('Calibrated') ))
    # d_input = int(raw_input('[1-4]  '))
    d_input = int(input('[1-4]  '))
    
    print('\n Select timeframe:  ')
    print(str('{:<2d}'.format(1) + '{:<20}'.format('Year')))
    print(str('{:<2d}'.format(2) + '{:<20}'.format('Month')))
    print(str('{:<2d}'.format(3) + '{:<20}'.format('All Time')))
    print(str('{:<2d}'.format(4) + '{:<20}'.format('Last Week')))
    # t_input = int(raw_input('[1-4]  '))
    t_input = int(input('[1-4]  '))
    
    # all_q = raw_input('\n All devices? (y/n) ')
    all_q = input('\n All devices? (y/n) ')
    if all_q == 'y':
        for i in range(len(device_ids)):
            if t_input == 1:
                if d_input == 1:
                    write_csv_file(y_start[i], y_end[i],      "daily", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(y_start[i], y_end[i],     "hourly", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(y_start[i], y_end[i],    "aux_raw", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(y_start[i], y_end[i], "calibrated", y_start_fn[i], y_end_fn[i], device_ids[i])
            if t_input == 2:
                if d_input == 1:
                    write_csv_file(m_start[i], m_end[i],      "daily", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(m_start[i], m_end[i],     "hourly", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(m_start[i], m_end[i],    "aux_raw", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(m_start[i], m_end[i], "calibrated", m_start_fn[i], m_end_fn[i], device_ids[i])
            if t_input == 3:
                if d_input == 1:
                    write_csv_file(astart[i], end[i],      "daily", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(astart[i], end[i],     "hourly", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(astart[i], end[i],    "aux_raw", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(astart[i], end[i], "calibrated", start_fn[i], end_fn[i], device_ids[i])
            if t_input == 4:
                if d_input == 1:
                    write_csv_file(stawk, endwk,      "daily", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 2:
                    write_csv_file(stawk, endwk,     "hourly", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 3:
                    write_csv_file(stawk, endwk,    "aux_raw", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 4:
                    write_csv_file(stawk, endwk, "calibrated", stawk_fn, endwk_fn, device_ids[i])                    
                    
    elif all_q == 'n':
        print('\n Select device: ')
        print('-----------------------------------')
        print(str('{:6}'.format('Number')) + '  ' + '{:<7}'.format('ID') + '  ' + '{:<10}'.format('Location Name'))
        print('-----------------------------------')
        for i in range(len(device_ids)):
            print(str('{:<6d}'.format(i+1)) + '  ' + '{:^7}'.format(str(device_ids[i]).strip("[' ']")) + '  ' + '{:^10}'.format(str(device_names[i]).strip("['Z ']")))
            print('-----------------------------------')
        # j = int(raw_input('[1-' + str(len(device_ids)) + ']  '))
        j = int(input('[1-' + str(len(device_ids)) + ']  '))
        if 1 <= j <= int(len(device_ids)):
            i = j-1
            if t_input == 1:
                if d_input == 1:
                    write_csv_file(y_start[i], y_end[i],      "daily", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(y_start[i], y_end[i],     "hourly", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(y_start[i], y_end[i],    "aux_raw", y_start_fn[i], y_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(y_start[i], y_end[i], "calibrated", y_start_fn[i], y_end_fn[i], device_ids[i])
            if t_input == 2:
                if d_input == 1:
                    write_csv_file(m_start[i], m_end[i],      "daily", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(m_start[i], m_end[i],     "hourly", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(m_start[i], m_end[i],    "aux_raw", m_start_fn[i], m_end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(m_start[i], m_end[i], "calibrated", m_start_fn[i], m_end_fn[i], device_ids[i])
            if t_input == 3:
                if d_input == 1:
                    write_csv_file(astart[i], end[i],      "daily", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 2:
                    write_csv_file(astart[i], end[i],     "hourly", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 3:
                    write_csv_file(astart[i], end[i],    "aux_raw", start_fn[i], end_fn[i], device_ids[i])
                elif d_input == 4:
                    write_csv_file(astart[i], end[i], "calibrated", start_fn[i], end_fn[i], device_ids[i])
            if t_input == 4:
                if d_input == 1:
                    write_csv_file(stawk, endwk,      "daily", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 2:
                    write_csv_file(stawk, endwk,     "hourly", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 3:
                    write_csv_file(stawk, endwk,    "aux_raw", stawk_fn, endwk_fn, device_ids[i])
                elif d_input == 4:
                    write_csv_file(stawk, endwk, "calibrated", stawk_fn, endwk_fn, device_ids[i])
        else:
            print('User input error')
    
    else:
        print('User input error')
elif f_input == 'n':
    print('\n No files written.')
else:
    print('User input error')
    
items = client.folder(folder_id = folder_id).get_items()
for item in items:
    print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))

In [ ]:
# Make temperature charts for specific timeframe
charts = []
for i in range(len(device_names)):
    de = arable_query('all', 'csv', device_ids[i], "aux_raw", "time", (m_end[i])[-1], (m_start[i])[-1])
    de['time'] = pd.to_datetime(de['time'],infer_datetime_format=True)
    base = alt.Chart(de, title=str((device_names[i])[2:]) + ' \n ' + str(((m_start[i])[-1])[:10]) + ' -- ' + str((m_end[i])[-1])[:10]
    ).mark_line(
        color='black'
    ).properties(
        width=800,
        height=100
    ).encode(
        x=alt.X('time:T', axis=alt.Axis(title='')),
        y=alt.Y('analog:Q', axis=alt.Axis(title='Analog')))
    z = alt.Chart(de, 
    ).mark_line(
        color='blue'
    ).properties(
        width=800,
        height=80
    ).encode(
        x=alt.X('time:T', axis=alt.Axis(title='')),
        y=alt.Y('sdi12_value_0:Q', axis=alt.Axis(title='SD112 Value 0')))
    q = alt.Chart(de, 
    ).mark_line(
        color='lightblue'
    ).properties(
        width=800,
        height=80
    ).encode(
        x=alt.X('time:T', axis=alt.Axis(title='')),
        y=alt.Y('sdi12_value_2:Q', axis=alt.Axis(title='SD112 Value 2')))
    graph = alt.vconcat(base, z, q)
    charts.append(graph)
alt.vconcat(*charts)

In [ ]:
df = StringIO(df)
df = pd.read_csv(df, sep=',', error_bad_lines=False)

fig, ax = plt.subplots(figsize=(3, 3))
ax = plt.gca()

df.plot(kind='line',x='time',y='Tabove',ax=ax)
df.plot(kind='line',x='time',y='Tbelow', color='red', ax=ax)
df.plot(kind='line',x='time',y='Tair', color='green', ax=ax)
df.plot(kind='line',x='time',y='Tdew', color='blue', ax=ax)
plt.xlabel('')
plt.ylabel('Deg. C')
plt.title(str((device_names[0])[1:]) + ' \n ' + str(((sta_m[0])[0])[:10]) + ' -- ' + str((end_m[0])[0])[:10])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()